<a href="https://colab.research.google.com/github/adsferreira/convnet.mnist/blob/main/cnn_mnist_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importando pacotes do Keras:

In [ ]:
from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical

Criando as camadas convolucionais da rede neural artificial, intercaladas com camadas do tipo max pooling 2D:


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Vale notar que uma *convnet* recebe, como entrada, tensores de dimensão *(altura_imagem, largura_imagem, canal)*. Neste caso, configuramos uma *convnet* para processar, como entrada, imagens de dimensãão (28, 28, 1), que é o formato das imagens presentes na base MNIST. Ou seja, estas imagens possuem 28 pixels de altura e largura, e canais com apenas um componente, que é típico de imagens cujos pixels são colorizados em escala de cinza.  

Agora, podemos imprimir a arquitetura da *convnet* que configuramos até agora:

In [ ]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Podemos notar, pela saída acima, que a saída de cada camada *Conv2D* e *MaxPooling2D* é um tensor na forma *(altura, largura, canal)*. As alturas e larguras de cada camada tendem a diminuir conforme a rede neural vai ficando mais profunda (adquiri mais camadas). O número de canais é controlado pelo primeiro argumento passado às camadas *Conv2D*: 32 ou 64 nós. 

O próximo passo é incluir um classificador densamente conectado na saída da parte convolucional da rede neural. Isto é feito criando uma pilha de camadas do tipo *Dense*. Este tipo de camada processa vetores 1D, enquanto a saída da última camada convolucional é um tensor 3D de dimensão (3, 3, 64), como mostra o sumário da arquitetura acima. Desta forma, primeiramente linearizamos o tensor 3D, de forma que se torne um vetor 1D. Para isso, incluímos uma camada *flatten* utilizando o méétodo *Flatten()* do objeto *layers*:

In [ ]:
model.add(layers.Flatten())

Agora, incluimos algumas camadas do tipo *Dense* no topo da rede neural:

In [ ]:
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Notem que a última camada possui 10 nós. Esta configuração é necessária, pois existem 10 diferentes classes na base MNIST. Desta forma, um determinado nó na saída deverá ser *mais ativado* quando a rede processar uma imagem que corresponde a sua respectiva classe.

Conferindo, novamente, se a *convnet* está projetada assim como determinamos no código acima:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

Como podemos notar, a saída da última camada convolucional, cuja dimensões são (3, 3, 64), foram linearizadas para um vetor 1D de 576 elementos (3 x 3 x 64 = 576?):

In [ ]:
print(3 * 3 * 64)

576


Depois de configurar o modelo, vamos treiná-lo na base do MINST. Primeiramente, carregamos as imagens, que já estão disponíveis no *keras*:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Vamos conhecer as dimensões das imagens de entrada e rótulos presente na base MNIST:

In [ ]:
print(train_images.shape)

(60000, 28, 28)


Podemos notar que o conjunto de dados de treinamento possui 60000 imagens de 28x28 pixels. Devemos recordar, no entanto, que o *keras* espera, como entrada, imagens com dimensões *(altura_imagem, largura_imagem, canais)*. Como as imagens na base MNIST estão organizadas com as dimensões de altura e largura apenas, devemos reorganizá-las para também possuírem a dimensão *canais*. Isso pode ser facilmente feito utilizando a função *reshape* disponível no pacote *numpy*:

In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

Notem, acima, que as imagens de teste também precisam apresentar a dimensão *canais*.

Podemos checar também o tipo de dado dos elementos da imagem. Para isso, é suficiente checar o tipo de dado do primeiro elemento da matriz *train_images*:

In [ ]:
print(type(train_images[0][0][0][0]))

<class 'numpy.uint8'>


Podemos notar que os elementos da imagem são inteiros de 8 bits sem sinal. De fato, esperamos que os elementos sejam do tipo inteiro. Imprimimos a última dimensão da matriz, pois é na dimensão *canais* que aparecem os valores que definem as cores em cada pixel da imagem. No caso das imagens da base MNIST, as cores são cinzas, que são valores escalares inteiros (precisam de apenas um canal) que variam de 0 (preto) a 255 (branco). No caso de imagens coloridas, é típico utilizar-se a representação RGB (Red-Green-Blue), que necessitam de três canais para representação (um para cada cor, vermelha, verde e azul), e cada canal poderá assumir uma intensidade que varia de 0 a 255.


Podemos checar o menor e o maior valor presente em todas imagens da base MNIST. Para isso, utilizaremos as funções *min* e *max* do *numpy*:

In [ ]:
print("valor mínimo encontrado nas imagens da base de treinamento: ", train_images.min())
print("valor máximo encontrado nas imagens da base de treinamento: ", train_images.max())
print("valor mínimo encontrado nas imagens da base de teste: ", test_images.max())
print("valor máximo encontrado nas imagens da base de teste: ", test_images.max())

valor mínimo encontrado nas imagens da base de treinamento:  0
valor máximo encontrado nas imagens da base de treinamento:  255
valor mínimo encontrado nas imagens da base de teste:  255
valor máximo encontrado nas imagens da base de teste:  255


Podemos afirmar que ao menos uma imagem possui as cores preto e branco na base MNIST, pois podemos afirmar que a base possui o valor 0 e 255 para alguns pixels das imagens.

No caso de Redes Neurais Artificiais, é comum normalizarmos os dados de entrada antes do processamento pela rede. Por exemplo, números grandes, como o 255 para representar uma cor, podem ser inadequados para minimização da função custo, pois podem ir para os pontos de saturação das funções de ativação ou gerar gradientes grandes que podem não se adaptar as regiões de minimização. Assim, um tipo de normalização simples é escalar os valores de entrada para o intervalo 0-1, conforme abaixo:

In [ ]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

Notem que os tipos agora devem ser valores reais. Assim, os convertemos para o tipo *float32*.

Não diferente, precisamos também adaptar os rótulos. Primeiramente, vamos checar as dimensões dos rótulos presentes na base MNIST:

In [ ]:
print(train_labels.shape)
print(test_labels.shape)

(60000,)
(10000,)


Podemos notar que ambos são vetores (de uma dimensão), e que cada vetor possui uma quantidade de rótulos correspondente com o número de imagens em cada conjunto de dados respectivo. De fato, os rótulos são os números inteiros representados por cada imagem, ou seja, os decimais de 0 a 9. Podemos utilizar as funções *min* e *max* para validarmos os valores mínimo e máximo presentes nos vetores acima: 

In [ ]:
print("menor valor encontrado nos rótulos de treinamento", train_labels.min())
print("maior valor encontrado nos rótulos de treinamento", train_labels.max())
print("menor valor encontrado nos rótulos de teste", test_labels.min())
print("maior valor encontrado nos rótulos de teste", test_labels.max())

menor valor encontrado nos rótulos de treinamento 0
maior valor encontrado nos rótulos de treinamento 9
menor valor encontrado nos rótulos de teste 0
maior valor encontrado nos rótulos de teste 9


No caso de classificadores (como o que estamos desenvolvendo aqui), a camada de saída tipicamente possuí um número de neurônios de saída igual ao número de classes presentes no problema. Como definido na nossa arquitetura, colocamos 10 neurônios na última camada, pois temos um total de 10 classes (decimais de 0 a 9).

Há um motivo simples para arquitetar a saída da rede desta maneira: queremos modelar a rede de forma que quando esta receba uma imagem com o caracter zero desenhado, seu primeiro neurônio de saída ative mais que o restante dos neurônios de saída, ou seja, o neurônio que ativou responda com saída 1 e o restante com saída 0, e assim por diante. Assim, vamos mudar a representação dos rótulos: ao invés de utilizar apenas o número presente no rótulo, faremos com que esse número seja representado por um vetor de 10 posições (pois temos 10 classes), e que este vetor contenha todos elementos com valores zero, exceto o elemento que está na posição representada pelo respectivo número. Este elemento recebe o valor 1. Por exemplo, um rótulo com valor 9 será representado por um vetor de zeros em todas posições, exceto na posição 9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1].

De fato, a intenção é modelarmos a rede para que seu último neurônio de saída dê como resposta o valor 1, e todo o restante responda com zero.

Como este tipo de mudança é bem típico, o *keras* já provê esta funcionalidade através da função *to_categorical*:

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Com a arquitetura da rede e base de dados configuradas, resta-nos apenas configurar como se dará o treinamento e executá-lo. Os elementos básicos que configuraremos são o otimizador que realizará a modelagem (algoritmo de treinamento em si), o tipo de função custo (*loss function*) a analisar, e a métrica de desempenho usada como referência. O otimizador usado será o *rmsprop*, a função custo será a *categorical_crossentropy* (tipicamente usada para classificadores multi-classes), e a métrica será a acurácia de resposta do modelo. Esta configuração é realizada pela função *compile* do objeto *model*: 

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Agora podemos realizar o treinamento para obtenção do modelo de predição de caracteres. Para isso, o *keras* ajustará o modelo para criar a relação entre imagens e os respectivos rótulos de treinamento, através da função *fit* do objeto *model*. Precisamos indicar quantas épocas (iterações) a rede será treinada, assim como o número de amostras usadas para validação do modelo:

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1815 - accuracy: 0.9427
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0482 - accuracy: 0.9848
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0327 - accuracy: 0.9894
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0248 - accuracy: 0.9923
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0193 - accuracy: 0.9941


Na saída gerada durante o treinamento, acima, temos os valores tanto da função custo quanto da acurácia ao longo das épocas. É possível notar que ambos valores caem ao longo do treinamento, de forma que a rede melhore seu desempenho ao longo do treinamento.

Mas a validação a respeito da qualidade do modelo só ocorre ao analisarmos seu desempenho sobre o conjunto de teste, pois estes dados são desconhecidos para a rede (nunca foram apresentado para teste modelo durante o treinamento). Abaixo, checamos o desempenho com a função *evaluate* do objeto *model*:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("valor da função custo no conjunto de testes: ", test_loss)
print("acurácia do modelo no conjunto de testes: ", test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0262 - accuracy: 0.9921
valor da função custo no conjunto de testes:  0.02624332904815674
acurácia do modelo no conjunto de testes:  0.9921000003814697


Como podemos notar, nossa rede tem acurácia de aproximadamente 0.99 no conjunto de teste, o que significa que o modelo, ao processar 10000 imagens de testes, classificou corretamente aproximadamente 99% das imagens.